# 04. Submission и AutoReport для VK EdTech ML Challenge

В этом ноутбуке:
* загружаем предсказания моделей;
* формируем файл `submission.tsv` нужного формата;
* запускаем `metrics.py` для офлайн-оценки;
* создаём текстовый AutoReport.


# ЭТАП 1. Импорт и загрузка предсказаний

In [13]:
from google.colab import drive
drive.mount("/content/drive")

import os
import subprocess
import datetime
import warnings

import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

FOLDER = "/content/drive/MyDrive/VK_Project_v2"

preds_path = os.path.join(FOLDER, "preds_validate.parquet")
preds_df = pd.read_parquet(preds_path)

print("preds_df.shape:", preds_df.shape)
display(preds_df.head())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
preds_df.shape: (1008, 3)


,at_least_one,at_least_two,at_least_three
0,0.020000,0.010000,5.000000e-03
1,0.022350,0.000286,1.734723e-18
2,0.104817,0.018036,1.000000e-04
3,0.210367,0.106725,5.915806e-02
4,0.359300,0.267008,1.819818e-01


# ЭТАП 2. Формирование submission.tsv

In [14]:
target_cols = ["at_least_one", "at_least_two", "at_least_three"]

submission = preds_df[target_cols].copy()

print("submission.shape:", submission.shape)
display(submission.head())

submission_path = os.path.join(FOLDER, "submission.tsv")
submission.to_csv(submission_path, sep="\t", index=False)
print("submission.tsv сохранён в", submission_path)

# Проверка повторного чтения
submission_check = pd.read_csv(submission_path, sep="\t")
print("Проверка чтения submission.tsv:", submission_check.shape)

submission.shape: (1008, 3)


,at_least_one,at_least_two,at_least_three
0,0.020000,0.010000,5.000000e-03
1,0.022350,0.000286,1.734723e-18
2,0.104817,0.018036,1.000000e-04
3,0.210367,0.106725,5.915806e-02
4,0.359300,0.267008,1.819818e-01


submission.tsv сохранён в /content/drive/MyDrive/VK_Project_v2/submission.tsv
Проверка чтения submission.tsv: (1008, 3)


# ЭТАП 3. Подготовка к запуску metrics.py

In [15]:
metrics_path = os.path.join(FOLDER, "metrics.py")
answers_path = os.path.join(FOLDER, "validate_answers.tsv")

print("metrics.py:", metrics_path)
print("answers:", answers_path)
print("submission:", submission_path)

metrics.py: /content/drive/MyDrive/VK_Project_v2/metrics.py
answers: /content/drive/MyDrive/VK_Project_v2/validate_answers.tsv
submission: /content/drive/MyDrive/VK_Project_v2/submission.tsv


# ЭТАП 4. Запуск metrics.py для оценки SMLogAccRatio

In [16]:
cmd = [
    "python",
    metrics_path,
    answers_path,
    submission_path,
]

print("Запускаем:", " ".join(cmd))
result = subprocess.run(cmd, capture_output=True, text=True)

print("stdout:")
print(result.stdout)
print("stderr:")
print(result.stderr)

import re
match = re.search(r"([-+]?\d*\.\d+|\d+)", result.stdout)
score = float(match.group(0)) if match else None
print("SMLogAccRatio (parsed):", score)

Запускаем: python /content/drive/MyDrive/VK_Project_v2/metrics.py /content/drive/MyDrive/VK_Project_v2/validate_answers.tsv /content/drive/MyDrive/VK_Project_v2/submission.tsv
stdout:
49.12

stderr:

SMLogAccRatio (parsed): 49.12
